## Simple RNN

In [1]:
import numpy as np


timesteps = 100
input_features = 32
output_features = 64

inputs = np.random.random((timesteps, input_features))
state_t = np.zeros(shape=(output_features, ))


W = np.random.random((output_features, input_features))
U = np.random.random((output_features, output_features))
b = np.random.random((output_features, ))

successive_outputs = []

for input_t in inputs:
    output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b)
    state_t = output_t
    successive_outputs.append(output_t)
    
final_output_sequence = np.vstack(successive_outputs)

In [2]:
final_output_sequence.shape

(100, 64)

## Simple RNN From Keras

In [3]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN


vocabulary_sz = 10000
embedding_sz = 32

model = Sequential()
model.add(Embedding(input_dim=vocabulary_sz, output_dim=embedding_sz))
model.add(SimpleRNN(32))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          320000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                2080      
Total params: 322,080
Trainable params: 322,080
Non-trainable params: 0
_________________________________________________________________
